In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data


/Users/charming/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [25]:
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [45]:
tf.reset_default_graph()

In [91]:
learning_rate = 0.001
training_epoch = 15
batch_size = 100

In [92]:
tf.set_random_seed(777)


In [48]:
class Model:
    
    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
        self._build_net()
    
    def _build_net(self):
        with tf.variable_scope(self.name):
            self.keep_prob = tf.placeholder(tf.float32)
            
            self.X = tf.placeholder(tf.float32,shape=[None,784])
            X_img = tf.reshape(self.X,[-1,28,28,1])
            self.Y = tf.placeholder(tf.float32,shape=[None,10])
            
            W1 = tf.Variable(tf.random_normal([3,3,1,32],stddev=0.01))
            L1 = tf.nn.conv2d(X_img,W1,strides=[1,1,1,1],padding="SAME")
            L1 = tf.nn.relu(L1)
            L1 = tf.nn.max_pool(L1,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")
            L1 = tf.nn.dropout(L1,keep_prob=self.keep_prob)
            
            W2 = tf.Variable(tf.random_normal([3,3,32,64],stddev=0.01))
            L2 = tf.nn.conv2d(L1,W2,strides=[1,1,1,1],padding="SAME")
            L2 = tf.nn.relu(L2)
            L2 = tf.nn.max_pool(L2,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")
            L2 = tf.nn.dropout(L2, keep_prob=self.keep_prob)
            
            W3 = tf.Variable(tf.random_normal([3,3,64,128],stddev=0.01))
            L3 = tf.nn.conv2d(L2,W3,strides=[1,1,1,1],padding="SAME")
            L3 = tf.nn.relu(L3)
            L3 = tf.nn.max_pool(L3,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")
            L3 = tf.nn.dropout(L3,keep_prob=self.keep_prob)
            L3_flat = tf.reshape(L3,[-1,4*4*128])
            
            W4 = tf.get_variable("W4",shape=[4*4*128,625],
                                initializer=tf.contrib.layers.xavier_initializer())
            b4 = tf.Variable(tf.random_normal([625]))
            L4 = tf.nn.relu(tf.matmul(L3_flat,W4)+b4)
            L4 = tf.nn.dropout(L4, keep_prob=self.keep_prob)
            
            W5 = tf.get_variable("W5",shape=[625,10],
                                initializer = tf.contrib.layers.xavier_initializer())
            b5 = tf.Variable(tf.random_normal([10]))
            self.logits = tf.matmul(L4,W5) + b5
            
            self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=self.logits,labels=self.Y))
            self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.cost)
            
            pred = tf.equal(tf.argmax(self.logits,1),tf.argmax(self.Y,1))
            self.accuracy = tf.reduce_mean(tf.cast(pred,tf.float32))
            
    def predict(self,x_test,keep_prob=1.0):
        return self.sess.run(self.accuracy,feed_dict={self.X:x_test,self.keep_prob:keep_prob})
    
    def get_accuracy(self,x_test,y_test,keep_prob=1.0):
        return self.sess.run(self.accuracy, feed_dict={self.X:x_test,self.Y:y_test,self.keep_prob:keep_prob})
    def train(self,x_data,y_data,keep_prob=0.7):
        return self.sess.run([self.cost,self.optimizer],feed_dict={self.X:x_data,self.Y:y_data,self.keep_prob:keep_prob})
            

In [49]:
sess = tf.Session()
m1 = Model(sess,"m1")

In [50]:
sess.run(tf.global_variables_initializer())

In [51]:
print("학습시작")

for epoch in range(training_epoch):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)
    
    for i in range(total_batch):
        batch_xs,batch_ys = mnist.train.next_batch(batch_size)
        c,_ = m1.train(batch_xs,batch_ys)
        avg_cost += c / total_batch
    print("Epoch :","%04d" % (epoch+1),"cost =",'{:.9f}'.format(avg_cost))
    
print("학습 끝!")

학습시작
Epoch : 0001 cost = 0.370469792
Epoch : 0002 cost = 0.099917548
Epoch : 0003 cost = 0.074749889
Epoch : 0004 cost = 0.060318654
Epoch : 0005 cost = 0.055316245
Epoch : 0006 cost = 0.048131594
Epoch : 0007 cost = 0.043621943
Epoch : 0008 cost = 0.041317780
Epoch : 0009 cost = 0.036686370
Epoch : 0010 cost = 0.036495511
Epoch : 0011 cost = 0.032111021
Epoch : 0012 cost = 0.032669552
Epoch : 0013 cost = 0.029420214
Epoch : 0014 cost = 0.025752983
Epoch : 0015 cost = 0.026324516
학습 끝!


In [52]:
print("Accuracy :",m1.get_accuracy(mnist.test.images,mnist.test.labels) )

Accuracy : 0.9934


In [99]:
tf.reset_default_graph()

In [100]:
class Model:
    
    def __init__(self,sess,name):
        self.sess = sess
        self.name = name
        self._build_net()
        
    def _build_net(self):
        with tf.variable_scope(self.name):
        
            self.training = tf.placeholder(tf.bool)
        
            self.X = tf.placeholder(tf.float32,shape=[None,784])
            X_img = tf.reshape(self.X,[-1,28,28,1])
            self.Y = tf.placeholder(tf.float32,shape=[None,10])
        
            conv1 = tf.layers.conv2d(inputs=X_img,filters=32,kernel_size=[3,3],padding='SAME',activation=tf.nn.relu)
            pool1 = tf.layers.max_pooling2d(inputs=conv1,pool_size=[2,2], padding="SAME",strides=2)
            dropout1 = tf.layers.dropout(inputs=pool1,rate=0.3,training=self.training)
        
            conv2 = tf.layers.conv2d(inputs=dropout1, filters=64,kernel_size=[3,3],padding="SAME",activation=tf.nn.relu)
            pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2,2],padding="SAME",strides=2)
            dropout2 = tf.layers.dropout(inputs=pool2,rate=0.3,training=self.training)
        
            conv3 = tf.layers.conv2d(inputs=dropout2,filters=128,kernel_size=[3,3],padding="SAME",activation=tf.nn.relu)
            pool3 = tf.layers.max_pooling2d(inputs=conv3,pool_size=[2,2],padding="SAME",strides=2)
            dropout3 = tf.layers.dropout(pool3,rate=0.3,training=self.training)
        
            flat = tf.reshape(dropout3,[-1,4*4*128])
            dense4 = tf.layers.dense(inputs=flat,units=625,activation=tf.nn.relu)
            dropout4 = tf.layers.dropout(dense4,rate=0.5,training=self.training)
        
            self.logits = tf.layers.dense(inputs=dropout4,units=10)
        
            self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=self.logits,labels=self.Y))
            self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.cost)
        
            self.prediction = tf.equal(tf.argmax(self.logits,1),tf.argmax(self.Y,1))
            self.accuracy = tf.reduce_mean(tf.cast(self.prediction,tf.float32))
        
    def predict(self,x_test,training=False):
        return self.sess.run(self.logits,feed_dict={self.X:x_test,self.training:training})
    
    def get_accuracy(self,x_test,y_test,training=False):
        return self.sess.run(self.accuracy,feed_dict={self.X:x_test,self.Y:y_test,self.training:training})
    
    def train(self,x_data,y_data,training=True):
        return self.sess.run([self.cost,self.optimizer],feed_dict={self.X:x_data,self.Y:y_data,self.training:training})

In [94]:
sess = tf.Session()
m1 = Model(sess,"m1")
sess.run(tf.global_variables_initializer())

In [95]:
print("학습시작")
for epoch in range(training_epoch):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        c,_ = m2.train(batch_xs,batch_ys)
        avg_cost += c / total_batch
        
    print("Epoch :","%04d" % (epoch + 1),'cost =','{:9f}'.format(avg_cost))
    
print('learning finished')

학습시작
Epoch : 0001 cost =  0.279700
Epoch : 0002 cost =  0.087130
Epoch : 0003 cost =  0.067808
Epoch : 0004 cost =  0.055849
Epoch : 0005 cost =  0.048996
Epoch : 0006 cost =  0.043687
Epoch : 0007 cost =  0.040355
Epoch : 0008 cost =  0.037427
Epoch : 0009 cost =  0.034828
Epoch : 0010 cost =  0.032536
Epoch : 0011 cost =  0.031610
Epoch : 0012 cost =  0.030099
Epoch : 0013 cost =  0.030560
Epoch : 0014 cost =  0.028340
Epoch : 0015 cost =  0.026711
learning finished


In [96]:
print("Accuracy :",m1.get_accuracy(mnist.test.images, mnist.test.labels))

Accuracy : 0.0817


In [101]:
sess = tf.Session()

models = []
num_models = 2
for m in range(num_models):
    models.append(Model(sess, "model" + str(m)))

sess.run(tf.global_variables_initializer())

print('Learning Started!')

# train my model
for epoch in range(training_epoch):
    avg_cost_list = np.zeros(len(models))
    total_batch = int(mnist.train.num_examples / batch_size)
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)

        # train each model
        for m_idx, m in enumerate(models):
            c, _ = m.train(batch_xs, batch_ys)
            avg_cost_list[m_idx] += c / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', avg_cost_list)

print('Learning Finished!')

# Test model and check accuracy
test_size = len(mnist.test.labels)
predictions = np.zeros([test_size, 10])
for m_idx, m in enumerate(models):
    print(m_idx, 'Accuracy:', m.get_accuracy(
        mnist.test.images, mnist.test.labels))
    p = m.predict(mnist.test.images)
    predictions += p

ensemble_correct_prediction = tf.equal(
    tf.argmax(predictions, 1), tf.argmax(mnist.test.labels, 1))
ensemble_accuracy = tf.reduce_mean(
    tf.cast(ensemble_correct_prediction, tf.float32))
print('Ensemble accuracy:', sess.run(ensemble_accuracy))

Learning Started!
Epoch: 0001 cost = [0.29412143 0.27615344]
Epoch: 0002 cost = [0.08701184 0.08732817]
Epoch: 0003 cost = [0.06594167 0.06543173]
Epoch: 0004 cost = [0.05547602 0.05589784]
Epoch: 0005 cost = [0.04907913 0.04861948]
Epoch: 0006 cost = [0.04404677 0.04408106]
Epoch: 0007 cost = [0.04039562 0.04109624]
Epoch: 0008 cost = [0.03703851 0.03701658]
Epoch: 0009 cost = [0.03506698 0.03750429]
Epoch: 0010 cost = [0.03212092 0.03181282]
Epoch: 0011 cost = [0.03149052 0.03145641]
Epoch: 0012 cost = [0.03084529 0.03127009]
Epoch: 0013 cost = [0.02920732 0.02892722]
Epoch: 0014 cost = [0.02745595 0.02761675]
Epoch: 0015 cost = [0.0264785  0.02737121]
Learning Finished!
0 Accuracy: 0.9938
1 Accuracy: 0.9949
Ensemble accuracy: 0.9956


In [103]:
sess = tf.Session()

models1 = []
num_models1 = 2
for m in range(num_models):
    models.append(Model(sess, "model" + str(m)))

sess.run(tf.global_variables_initializer())

print('Learning Started!')




In [112]:
test_size = len(mnist.test.labels)
print(test_size)
predictions = np.zeros([test_size, 10])
print(predictions)

10000
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [113]:
for m_idx, m in enumerate(models):
    print(m_idx, 'Accuracy:', m.get_accuracy(
        mnist.test.images, mnist.test.labels))
    p = m.predict(mnist.test.images)
    predictions += p

0 Accuracy: 0.9938
1 Accuracy: 0.9949


In [114]:
ensemble_correct_prediction = tf.equal(
    tf.argmax(predictions, 1), tf.argmax(mnist.test.labels, 1))
ensemble_accuracy = tf.reduce_mean(
    tf.cast(ensemble_correct_prediction, tf.float32))
print('Ensemble accuracy:', sess.run(ensemble_accuracy))

Ensemble accuracy: 0.9956
